In [1]:
try:
    import Image
except ImportError:
    from PIL import Image
import pytesseract
import webbrowser
import pyscreenshot as ImageGrab
from googleapiclient.discovery import build
import pprint
import cPickle as pickle
from pynput.keyboard import Key, Controller
import time
import re

In [2]:
my_api_key, my_cse_id = pickle.load(open('api_key_cse_id.p', 'rb'))
#to use Google Search custom API, you'll need your own API key and CSE ID.

def process(res_items):
    text_list = [re.sub(r'\n|,|:|\||\xa0|\s*\.+', '', item['title'] + ' ' + item['snippet']).lower() for item in res_items]
    return ' '.join(text_list)

def google_search(search_term, **kwargs):
    service = build("customsearch", "v1", developerKey=my_api_key)
    res = service.cse().list(q=search_term, cx=my_cse_id, **kwargs).execute()
    return process(res['items']), int(res['searchInformation']['formattedTotalResults'].replace(',', ''))

def print_results(results):
    for result in results:
        for key in result:
            print '%s: %s' % (key, result[key])
        print '\n\n'

In [3]:
print google_search('dogs', num=10)

(u"dog - wikipedia the domestic dog is a member of the genus canis (canines) which forms part of the wolf-like canids and is the most widely abundant terrestrial carnivore complete guide to caring for dogs  dog breed information dog your dog's online owners manual featuring articles about breed information dog selection training grooming and care for dogs and puppies funny dogs who don't want to take a bath - try not to laugh dec 10 2016 funny dog videos of funny dogs who don't want to take a bath or dogs and puppies who hate baths try not to laugh challenge - try not to laugh dog dog breeds adoption bringing a dog home and care everything you need to know about dogs including dog breeds how to adopt a dog bringing a dog home dog health and care and more! dogs dogs the adorable domestic pet beloved by millions the dog is one of the most popular pets in the world -- it is of course commonly referred to as \u201cman's best your d&d game doesn't have enough dogs in it - polygon 11 hours a

In [84]:
#Save for later: switch control

keyboard = Controller()
keyboard.press(' ')
keyboard.release(' ')

In [ ]:
''' coordinates for OCR text extraction from Reflector 3 window mirroring iPhone at "Fill Screen" scale,
positioned at (0, 0) on 13 inch MBA '''

question_coords = (55, 253, 376, 357)

answerX1, answerX2 = 75, 355
answerY1, answerY2 = 390, 415
offset = 63
answers_coords = [(answerX1, answerY1, answerX2, answerY2), 
                  (answerX1, answerY1 + offset, answerX2, answerY2 + offset),
                  (answerX1, answerY1 + 2*offset, answerX2, answerY2 + 2*offset)]

def parseQuestionAnswer():
    question_img=ImageGrab.grab(bbox=question_coords) # X1,Y1,X2,Y2
    answer_imgs = [ImageGrab.grab(bbox=coords) for coords in answers_coords]
    question = pytesseract.image_to_string(question_img).replace('\n', ' ').lower()
    answers = [pytesseract.image_to_string(img).lower() for img in answer_imgs]
    return question, answers

In [9]:
#every time a new question appears: 

'''
1. raw question
2. - 4. question + answers (no quotes)
'''
#For testing purposes
question = 'Which of these is a species of snake?'
answers = ['Ball python', 'Marbled salamander', 'Axolotl']

def getBestAnswer():
    #question, answers = parseQuestionAnswer()
    q_search = google_search(question, num=10)
    combine_queries = ['%s \"%s\"' % (question, answer) for answer in answers]
    combine_searches = [google_search(query, num=10) for query in combine_queries]
    answerCounts = [q_search[0].count(answer) for answer in answers]
    numSearchResults = [search[1] for search in combine_searches]

In [10]:
start = time.time()
getBestAnswer()
end = time.time()

print end - start

['Which of these is a species of snake? "Ball python"', 'Which of these is a species of snake? "Marbled salamander"', 'Which of these is a species of snake? "Axolotl"']
Ball python: 62300
Marbled salamander: 3380
Axolotl: 34200
best answer: Ball python
worst answer: Marbled salamander
3.21820902824


In [6]:
while True:
    raw_input('press any key to grab answer:')
    getBestAnswer()

KeyboardInterrupt: 